In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [4]:
df = pd.read_csv('output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_er = df_comuni[df_comuni['cod_regione'] == '08']

In [ ]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/votantiZ/DE/20241117/TE/07/PR/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_provincia']}/{row['tipo']}/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/votanti/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_81721/928981210.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_81721/928981210.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_comune,url_aff,url
172,080560560090,COLLECCHIO,CM,N,NaN,7,08,EMILIA-ROMAGNA,056,PARMA,0090,https://eleapi.interno.gov.it/siel/PX/votantiZ...,https://elezioni.interno.gov.it/risultati/2024...
16,080130130140,CASTEL DEL RIO,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,0140,https://eleapi.interno.gov.it/siel/PX/votantiZ...,https://elezioni.interno.gov.it/risultati/2024...
216,080610610060,BORGONOVO VAL TIDONE,CM,N,NaN,7,08,EMILIA-ROMAGNA,061,PIACENZA,0060,https://eleapi.interno.gov.it/siel/PX/votantiZ...,https://elezioni.interno.gov.it/risultati/2024...


In [27]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [2]:
print(f"Total URLs to process: {len(df_er['url_aff'])}")

comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
    print(f"\nProcessing record {idx + 1}")
    print(f"URL: {url}")

    response = requests.get(url, headers=headers)
    print(f"Response status code: {response.status_code}")
    
    data = response.json()
    
    provincia = df_er.iloc[idx]['provincia']
    regione = df_er.iloc[idx]['regione']
    url = df_er.iloc[idx]['url']
    print(f"Processing data for Provincia: {provincia}, Regione: {regione}")

    # Get total number of updates available
    n_updates = len(data['enti']['ente_p']['com_vot'])
    print(f"Number of updates available: {n_updates}")

    # Process each update
    for n_update in range(n_updates):
        raw_dt_com = str(data['enti']['ente_p']['com_vot'][n_update]['dt_com'])
        date = raw_dt_com[:8]
        time = raw_dt_com[8:12]

        formatted_date = f"{date[:4]}-{date[4:6]}-{date[6:]}"
        formatted_time = f"{time[:2]}:{time[2:]}"
        print(f"Processing update {n_update + 1}/{n_updates} - Timestamp: {formatted_date} {formatted_time}")

        comune_info = {
            'ele_info': data['int']['t_ele'],
            'comune': data['enti']['ente_p']['desc'],
            'provincia': provincia,
            'regione': regione,
            'ele_f': data['enti']['ente_p']['ele_f'],
            'ele_m': data['enti']['ente_p']['ele_m'],
            'ele_t': data['enti']['ente_p']['ele_t'],
            'perc': data['enti']['ente_p']['com_vot'][n_update]['perc'],
            'perc_r': data['enti']['ente_p']['com_vot'][n_update]['perc_r'],
            'vot_t': data['enti']['ente_p']['com_vot'][n_update]['vot_t'],
            'data': formatted_date,
            'ora': formatted_time,
            'update_number': n_update  # Added to track which update this is
        }
        
        print(f"Comune: {comune_info['comune']} - Update {n_update}")
        print(f"Electoral data - Total: {comune_info['ele_t']}, Female: {comune_info['ele_f']}, Male: {comune_info['ele_m']}")
        print(f"Voting percentage: {comune_info['perc']}%, Reporting percentage: {comune_info['perc_r']}%")

        comuni_data.append(comune_info)
    
    print(f"Waiting 0.2 seconds before next request...")
    sleep(0.2)

print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nConverting percentage columns to float...")
df_data['perc_r'] = df_data['perc_r'].str.replace(',', '.').astype(float)
df_data['perc'] = df_data['perc'].str.replace(',', '.').astype(float)

print("\nFinal DataFrame shape:", df_data.shape)
print("\nSample of processed data:")
print(df_data.head())

NameError: name 'df_er' is not defined

In [31]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,ele_info,comune,provincia,regione,ele_f,ele_m,ele_t,perc,perc_r,vot_t,data,ora
0,Regionali Speciali,Alto Reno Terme,Bologna,Emilia-Romagna,3057,2979,6036,26.34,60.27,1590,2024-11-17,19:00
1,Regionali Speciali,Anzola Dell'Emilia,Bologna,Emilia-Romagna,4867,4800,9667,37.73,65.89,3647,2024-11-17,19:00
2,Regionali Speciali,Argelato,Bologna,Emilia-Romagna,3959,3739,7698,37.36,65.89,2876,2024-11-17,19:00
3,Regionali Speciali,Baricella,Bologna,Emilia-Romagna,2700,2656,5356,31.57,60.58,1691,2024-11-17,19:00
4,Regionali Speciali,Bentivoglio,Bologna,Emilia-Romagna,2238,2233,4471,34.73,63.78,1553,2024-11-17,19:00
...,...,...,...,...,...,...,...,...,...,...,...,...
325,Regionali Speciali,Sant'Agata Feltria,Rimini,Emilia-Romagna,973,1000,1973,23.31,49.16,460,2024-11-17,19:00
326,Regionali Speciali,Santarcangelo Di Romagna,Rimini,Emilia-Romagna,9518,9004,18522,29.37,60.65,5440,2024-11-17,19:00
327,Regionali Speciali,Sassofeltrio,Rimini,Emilia-Romagna,746,755,1501,17.32,0.00,260,2024-11-17,19:00
328,Regionali Speciali,Talamello,Rimini,Emilia-Romagna,587,568,1155,22.68,51.29,262,2024-11-17,19:00


In [32]:
df_data['perc_diff'] = df_data['perc']-df_data['perc_r']
df_data.sample(5)

,ele_info,comune,provincia,regione,ele_f,ele_m,ele_t,perc,perc_r,vot_t,data,ora,perc_diff
148,Regionali Speciali,Sestola,Modena,Emilia-Romagna,1061,1090,2151,30.87,58.56,664,2024-11-17,19:00,-27.69
149,Regionali Speciali,Soliera,Modena,Emilia-Romagna,6269,6156,12425,32.52,62.02,4041,2024-11-17,19:00,-29.50
301,Regionali Speciali,Viano,Reggio Nell'Emilia,Emilia-Romagna,1408,1472,2880,30.03,57.27,865,2024-11-17,19:00,-27.24
157,Regionali Speciali,Bore,Parma,Emilia-Romagna,347,398,745,21.88,47.07,163,2024-11-17,19:00,-25.19
144,Regionali Speciali,San Prospero,Modena,Emilia-Romagna,2366,2399,4765,26.44,55.77,1260,2024-11-17,19:00,-29.33


In [33]:
df_data.to_csv('affluenze_er.csv', index=False, encoding='UTF-8-sig')